In [ ]:
import os
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
from google.colab.patches import cv2_imshow

def augment_image(image, bboxes, apply_grayscale=False, apply_color_adjustment=False):
    """이미지와 바운딩 박스를 증강하는 함수"""
    augmenters = [
        iaa.Fliplr(0.4),  # 40% 확률로 좌우 반전
        iaa.Flipud(0.4),  # 40% 확률로 상하 반전
        iaa.Crop(percent=(0, 0.2)),  # 이미지의 0%에서 20%를 자름
        iaa.Rotate((-45, 45)),  # -45도에서 45도 사이의 무작위 회전
        iaa.Add((-40, 40)),  # 밝기 변경
        iaa.Multiply((0.8, 1.2)),  # 대비 조절
        iaa.GaussianBlur(sigma=(0, 3.0)),  # 가우시안 블러
        iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}),  # 무작위로 이동
        iaa.AdditiveGaussianNoise(scale=(0, 0.1 * 255))
    ]

    if apply_grayscale:
        augmenters.append(iaa.Grayscale(alpha=1.0))  # 회색조로 변경

    if apply_color_adjustment:
        augmenters.append(iaa.Multiply((0.5, 1.5)))  # 색상 조정

    seq = iaa.Sequential(augmenters)

    # 이미지와 바운딩 박스에 증강 적용
    image_aug, bboxes_aug = seq(image=image, bounding_boxes=bboxes)

    return image_aug, bboxes_aug

def generate_multiple_augmented_images(image, bboxes, num_augmentations=20, num_gray_images=5, num_color_adjusted_images=5):
    """다양한 증강 이미지를 생성하는 함수"""
    augmented_images = []
    augmented_bboxes = []
    
    for i in range(num_augmentations):
        apply_grayscale = (i < num_gray_images)  # 처음 num_gray_images 만큼 회색조 적용
        apply_color_adjustment = (i < num_color_adjusted_images) and (i >= num_gray_images)  # 그 다음 num_color_adjusted_images 만큼 색상 조정 적용
        
        image_aug, bboxes_aug = augment_image(image, bboxes, apply_grayscale, apply_color_adjustment)
        
        # 바운딩 박스를 클리핑하여 이미지에 맞게 조정
        bboxes_aug_clipped = clip_bounding_boxes(bboxes_aug, image_aug.shape)
        augmented_images.append(image_aug)
        augmented_bboxes.append(bboxes_aug_clipped)
    
    return augmented_images, augmented_bboxes

def clip_bounding_boxes(bboxes, image_shape):
    """바운딩 박스를 이미지 경계 내로 클리핑하는 함수"""
    clipped_bboxes = []
    h, w = image_shape[:2]
    for bbox in bboxes.bounding_boxes:
        x1 = max(0, min(bbox.x1, w - 1))
        y1 = max(0, min(bbox.y1, h - 1))
        x2 = max(0, min(bbox.x2, w - 1))
        y2 = max(0, min(bbox.y2, h - 1))
        
        # 클리핑된 바운딩 박스만 추가
        if x1 < x2 and y1 < y2:  # 유효한 영역인지 확인
            clipped_bboxes.append(ia.BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2))

    return ia.BoundingBoxesOnImage(clipped_bboxes, shape=image_shape)

def visualize_augmented_images(images, bboxes_list):
    """증강된 이미지를 시각화하는 함수"""
    for img, bboxes in zip(images, bboxes_list):
        img_with_bboxes = img.copy()
        for bbox in bboxes.bounding_boxes:
            x1, y1, x2, y2 = int(bbox.x1), int(bbox.y1), int(bbox.x2), int(bbox.y2)
            cv2.rectangle(img_with_bboxes, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 바운딩 박스 그리기
        cv2_imshow(img_with_bboxes)  # 수정된 이미지 표시
        cv2.waitKey(0)  # 다음 이미지 표시를 위한 키 입력 대기

def load_bbox_data(file_path):
    """텍스트 파일에서 바운딩 박스 데이터를 읽어와 리스트에 저장하는 함수"""
    bbox_data = []
    
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            bbox_data.append((class_id, x_center, y_center, width, height))
    
    return bbox_data

def create_bounding_boxes(bbox_data, image_shape):
    """바운딩 박스를 imgaug 형식으로 변환하는 함수"""
    bboxes = []
    class_ids = []
    for (class_id, x_center, y_center, width, height) in bbox_data:
        x1 = (x_center - width / 2) * image_shape[1]
        y1 = (y_center - height / 2) * image_shape[0]
        x2 = (x_center + width / 2) * image_shape[1]
        y2 = (y_center + height / 2) * image_shape[0]
        
        bbox = ia.BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2)
        bboxes.append(bbox)
        class_ids.append(class_id)  # 클래스 ID 저장

    return ia.BoundingBoxesOnImage(bboxes, shape=image_shape), class_ids  # 클래스 ID 리스트 반환

def save_bbox_data(file_path, bboxes, class_ids, image_shape):
    """증강된 바운딩 박스를 텍스트 파일로 저장하는 함수"""
    h, w = image_shape[:2]
    with open(file_path, 'w') as file:
        for bbox, class_id in zip(bboxes.bounding_boxes, class_ids):  # bboxes와 class_ids를 함께 사용
            # 상대 좌표로 변환
            x_center = (bbox.x1 + bbox.x2) / (2 * w)
            y_center = (bbox.y1 + bbox.y2) / (2 * h)
            width = (bbox.x2 - bbox.x1) / w
            height = (bbox.y2 - bbox.y1) / h
            
            file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# 이미지와 바운딩 박스 로드
input_folder = "/content/drive/MyDrive/NIPA-Google/yolo_split/train"  # 원본 이미지 및 어노테이션이 있는 폴더 경로
output_folder = "/content/drive/MyDrive/NIPA-Google/yolo_split/train/augmented"  # 증강 이미지 저장 폴더 경로

# 출력 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# 모든 이미지에 대해 반복
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):  # JPEG 이미지 파일에 대해 작업
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        h, w = image.shape[:2]

        # 바운딩 박스 데이터 로드
        bbox_file_path = os.path.join(input_folder, f"{filename.split('.')[0]}.txt")
        bbox_data = load_bbox_data(bbox_file_path)

        # 바운딩 박스를 imgaug 형식으로 변환
        bboxes, class_ids = create_bounding_boxes(bbox_data, image.shape)

        # 여러 개의 증강 이미지 생성
        augmented_images, augmented_bboxes = generate_multiple_augmented_images(image, bboxes, num_augmentations=20, num_gray_images=5, num_color_adjusted_images=5)

        # 증강된 이미지 및 어노테이션 저장
        for i, (aug_img, aug_bboxes) in enumerate(zip(augmented_images, augmented_bboxes)):
            # 증강된 이미지 저장
            augmented_image_path = os.path.join(output_folder, f"{filename.split('.')[0]}_aug_{i}.jpg")
            cv2.imwrite(augmented_image_path, aug_img)

            # 어노테이션 파일 저장
            bbox_output_path = os.path.join(output_folder, f"{filename.split('.')[0]}_aug_{i}.txt")
            save_bbox_data(bbox_output_path, aug_bboxes, class_ids, aug_img.shape)  # 수정된 save_bbox_data 호출

print("증강 완료!")
